<img src="../code/Resources/cropped-SummerWorkshop_Header.png"> 

<h1 align="center">Workshop 1: Tutorial on neuronal encoding and behavior</h1> 
<h3 align="center">Summer Workshop on the Dynamic Brain</h3> 
<h3 align="center">Thursday, August 26th, 2025</h3> 
<h4 align="center">Day 2</h4> 

#### Add here description about motivation/task/data. Motivation to use dataset and point to databook for reference.  

<center><img src="Resources/DR_task_description.png", width="10300" height="400"> </center>

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm

import pynwb

pd.set_option('display.max_columns', None)

In [ ]:
# intialize session_id and get session object [may be use different mice per student?]

session_id = '742903_2024-10-22'

nwb_path = f'/root/capsule/data/{session_id}.nwb'
session = pynwb.NWBHDF5IO(nwb_path).read()

In [ ]:
# get trials data

trials = session.trials.to_dataframe()



In [ ]:
# get units table 

units_table = session.units.to_dataframe()

In [ ]:
def make_psth(spike_times, stim_times, pre_window=0.5, post_window=1.0, bin_size=0.05):
    """
    Generate a Peri-Stimulus Time Histogram (PSTH).
    
    Parameters:
    - spike_times: array-like, timestamps of all spikes (in seconds)
    - stim_times: array-like, timestamps of stimulus onsets (in seconds)
    - pre_window: float, time before stimulus to include in PSTH (seconds)
    - post_window: float, time after stimulus to include in PSTH (seconds)
    - bin_size: float, width of each time bin (seconds)
    
    Returns:
    - firing_rates: 2D numpy array of firing rates (trials x bins)
    - bin_centers: 1D numpy array of bin center times (relative to stimulus onset)
    """

    # Ensure inputs are numpy arrays 
    spike_times = np.array(spike_times)
    stim_times = np.array(stim_times)
    
    # Define bin edges from -pre_window to +post_window
    bins = np.arange(-pre_window, post_window + bin_size, bin_size)
    
    # Compute centers of bins (for plotting)
    bin_centers = bins[:-1] + bin_size / 2
    
    # Initialize a matrix to hold spike counts: rows = trials, columns = bins
    all_counts = np.zeros((len(stim_times), len(bins) - 1))
    
    # Loop through each stimulus time to compute trial-specific spike counts
    for i, stim_time in enumerate(stim_times):
        # Select spikes that fall within the time window around this stimulus
        mask = ((spike_times >= stim_time - pre_window) & 
                (spike_times < stim_time + post_window))
        
        # Align spike times to stimulus onset (0 = stimulus)
        trial_spikes = spike_times[mask] - stim_time
        
        # Bin the aligned spikes and count how many fall into each bin
        counts, _ = np.histogram(trial_spikes, bins=bins)
        
        # Store the result in the i-th row (trial)
        all_counts[i, :] = counts
    
    # Convert spike counts to firing rates (spikes per second)
    firing_rates = all_counts / bin_size
    
    # Return firing rates (trials x bins) and bin center positions
    return firing_rates, bin_centers
